In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stat
pd.set_option('display.max_columns', None)
%matplotlib inline

### Feature Processing
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from dateutil import relativedelta

### Feature Selection Techniques
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import VarianceThreshold

### Train Test Split
from sklearn.model_selection import train_test_split

### Model Building
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import  XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

### Metrics
import sklearn.metrics as metrics

### Hyper Paramter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV

### Loading and Saving Model
import pickle

### Ensemble Technique - Combining Multiple models
## We call the algorithms like this way

## Then we use Voting Classifier for ensemble learning:
from sklearn.ensemble import VotingClassifier

from keras import backend as K

import re

In [ ]:
train_bur_demo = pd.read_excel('/content/drive/MyDrive/LTFS_Top_up/train_bur_demo_mean_updated_string_manipulation.xlsx')
train_bur_demo.head()

,Unnamed: 0,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1,0,1,0,1,1,1,1,9,True,5,True False,PRIMARY,9,PRIMARY,4,Auto Loan (Personal),5,"['Auto Loan (Personal)', 'Credit Card', 'Overd...",2,NBF,5,"['NAB', 'NBF']",1,Individual,9,['Individual'],3,Closed,5,"['Active', 'Closed', 'Delinquent']",2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000
1,1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0,0,1,0,0,1,1,2,13,False,12,True False,PRIMARY,13,PRIMARY,7,Commercial Vehicle Loan,4,"['Auto Loan (Personal)', 'Business Loan Genera...",2,NBF,7,"['NBF', 'PRB']",2,Individual,9,"['Guarantor', 'Individual']",2,Active,7,"['Active', 'Closed']",1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188
2,2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6,0,0,1,1,0,1,3,31,False,29,True False,PRIMARY,31,PRIMARY,8,Gold Loan,8,"['Commercial Vehicle Loan', 'Consumer Loan', '...",5,PRB,13,"['CCC', 'COP', 'NAB', 'NBF', 'PRB']",3,Individual,26,"['Guarantor', 'Individual', 'Joint']",3,Closed,19,"['Active', 'Closed', 'Delinquent']",1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724
3,3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1,0,1,0,0,1,1,7,4,True,2,True False,PRIMARY,4,PRIMARY,2,Tractor Loan,3,"['Personal Loan', 'Tractor Loan']",1,NBF,4,['NBF'],1,Individual,4,['Individual'],2,Closed,2,"['Active', 'Closed']",3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000
4,4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2,0,1,0,1,1,1,8,7,False,6,True False,PRIMARY,7,PRIMARY,3,Tractor Loan,4,"['Commercial Vehicle Loan', 'Personal Loan', '...",3,NBF,4,"['NAB', 'NBF', 'PRB']",3,Individual,3,"['Guarantor', 'Individual', 'Joint']",4,Active,3,"['Active', 'Closed', 'Delinquent', 'SUIT FILED...",2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524


In [ ]:
train_bur_demo = train_bur_demo.iloc[:,1:]

In [ ]:
test_bur_demo=pd.read_excel('/content/drive/MyDrive/LTFS_Top_up/test_bur_demo_mean_updated_string_manipulation.xlsx')

In [ ]:
test_bur_demo = test_bur_demo.iloc[:,1:]

In [ ]:
display(train_bur_demo.head())
display(test_bur_demo.head())

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1,0,1,0,1,1,1,1,9,True,5,True False,PRIMARY,9,PRIMARY,4,Auto Loan (Personal),5,"['Auto Loan (Personal)', 'Credit Card', 'Overd...",2,NBF,5,"['NAB', 'NBF']",1,Individual,9,['Individual'],3,Closed,5,"['Active', 'Closed', 'Delinquent']",2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0,0,1,0,0,1,1,2,13,False,12,True False,PRIMARY,13,PRIMARY,7,Commercial Vehicle Loan,4,"['Auto Loan (Personal)', 'Business Loan Genera...",2,NBF,7,"['NBF', 'PRB']",2,Individual,9,"['Guarantor', 'Individual']",2,Active,7,"['Active', 'Closed']",1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6,0,0,1,1,0,1,3,31,False,29,True False,PRIMARY,31,PRIMARY,8,Gold Loan,8,"['Commercial Vehicle Loan', 'Consumer Loan', '...",5,PRB,13,"['CCC', 'COP', 'NAB', 'NBF', 'PRB']",3,Individual,26,"['Guarantor', 'Individual', 'Joint']",3,Closed,19,"['Active', 'Closed', 'Delinquent']",1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1,0,1,0,0,1,1,7,4,True,2,True False,PRIMARY,4,PRIMARY,2,Tractor Loan,3,"['Personal Loan', 'Tractor Loan']",1,NBF,4,['NBF'],1,Individual,4,['Individual'],2,Closed,2,"['Active', 'Closed']",3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2,0,1,0,1,1,1,8,7,False,6,True False,PRIMARY,7,PRIMARY,3,Tractor Loan,4,"['Commercial Vehicle Loan', 'Personal Loan', '...",3,NBF,4,"['NAB', 'NBF', 'PRB']",3,Individual,3,"['Guarantor', 'Individual', 'Joint']",4,Active,3,"['Active', 'Closed', 'Delinquent', 'SUIT FILED...",2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524


,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,4,False,3,True False,PRIMARY,4,PRIMARY,4,Tractor Loan,1,"['Credit Card', 'Housing Loan', 'Property Loan...",3,HFC,2,"['CCC', 'HFC', 'NBF']",2,Individual,2,"['Individual', 'Joint']",2,Active,3,"['Active', 'Closed']",741147.000000,8005.0,547818.500000,0.0,0.0,60.000000,1.403017,641864.840640,651.864532,0.0
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,8,False,7,True False,PRIMARY,8,PRIMARY,5,Gold Loan,3,"['Consumer Loan', 'Gold Loan', 'Housing Loan',...",4,NBF,3,"['HFC', 'MFI', 'NBF', 'PRB']",2,Individual,6,"['Individual', 'Joint']",2,Closed,6,"['Active', 'Closed']",242476.625000,1282.5,113403.875000,0.0,0.0,22.500000,2.127743,211590.023324,789.747186,0.0
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,12,False,10,True False,PRIMARY,12,PRIMARY,4,Auto Loan (Personal),7,"['Auto Loan (Personal)', 'Gold Loan', 'Persona...",2,NBF,11,"['NBF', 'PRB']",3,Individual,6,"['Guarantor', 'Individual', 'Joint']",2,Closed,11,"['Active', 'Closed']",269780.083333,0.0,8566.666667,0.0,0.0,14.083333,6.326831,66122.300453,1864.970541,0.0
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,2,True,1,True False,PRIMARY,2,PRIMARY,2,Business Loan Priority Sector Agriculture,1,"['Business Loan Priority Sector Agriculture',...",2,NBF,1,"['NAB', 'NBF']",1,Individual,2,['Individual'],2,Active,1,"['Active', 'Closed']",400000.000000,0.0,63796.500000,0.0,0.0,0.000000,0.000000,163930.644118,0.000000,0.0
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,2,True,1,True False,PRIMARY,2,PRIMARY,2,Auto Loan (Personal),1,"['Auto Loan (Personal)', 'Tractor Loan']",1,NBF,2,['NBF'],2,Guarantor,1,"['Guarantor', 'Individual']",2,Active,1,"['Active', 'Closed']",465000.000000,8000.0,58100.000000,0.0,0.0,23.000000,5.388889,253020.115873,158.800000,265000.0


In [ ]:
train_bur_demo['SELF-INDICATOR_Top_value'] = train_bur_demo['SELF-INDICATOR_Top_value'].astype('object')

In [ ]:
test_bur_demo['SELF-INDICATOR_Top_value'] = test_bur_demo['SELF-INDICATOR_Top_value'].astype('object')

In [ ]:
categorical_features = ['SELF-INDICATOR_Top_value', 'SELF-INDICATOR_unique_values', 'MATCH-TYPE_Top_value', 'MATCH-TYPE_unique_values',
                        'ACCT-TYPE_Top_value','ACCT-TYPE_unique_values','CONTRIBUTOR-TYPE_Top_value','CONTRIBUTOR-TYPE_unique_values',
                        'OWNERSHIP-IND_Top_value','OWNERSHIP-IND_unique_values','ACCOUNT-STATUS_Top_value','ACCOUNT-STATUS_unique_values']

### Cardinality of Categorical Variables
for feature in categorical_features:
  print('Feature : {} ------- Number of Unique Categories : {} '.format(feature,len(train_bur_demo[feature].unique())))

Feature : SELF-INDICATOR_Top_value ------- Number of Unique Categories : 2 
Feature : SELF-INDICATOR_unique_values ------- Number of Unique Categories : 3 
Feature : MATCH-TYPE_Top_value ------- Number of Unique Categories : 2 
Feature : MATCH-TYPE_unique_values ------- Number of Unique Categories : 3 
Feature : ACCT-TYPE_Top_value ------- Number of Unique Categories : 41 
Feature : ACCT-TYPE_unique_values ------- Number of Unique Categories : 7782 
Feature : CONTRIBUTOR-TYPE_Top_value ------- Number of Unique Categories : 10 
Feature : CONTRIBUTOR-TYPE_unique_values ------- Number of Unique Categories : 182 
Feature : OWNERSHIP-IND_Top_value ------- Number of Unique Categories : 5 
Feature : OWNERSHIP-IND_unique_values ------- Number of Unique Categories : 18 
Feature : ACCOUNT-STATUS_Top_value ------- Number of Unique Categories : 9 
Feature : ACCOUNT-STATUS_unique_values ------- Number of Unique Categories : 91 


In [ ]:
for feature in categorical_features:
  print('Feature : {} ------- Number of Unique Categories : {} '.format(feature,len(test_bur_demo[feature].unique())))

Feature : SELF-INDICATOR_Top_value ------- Number of Unique Categories : 2 
Feature : SELF-INDICATOR_unique_values ------- Number of Unique Categories : 3 
Feature : MATCH-TYPE_Top_value ------- Number of Unique Categories : 2 
Feature : MATCH-TYPE_unique_values ------- Number of Unique Categories : 3 
Feature : ACCT-TYPE_Top_value ------- Number of Unique Categories : 38 
Feature : ACCT-TYPE_unique_values ------- Number of Unique Categories : 1743 
Feature : CONTRIBUTOR-TYPE_Top_value ------- Number of Unique Categories : 9 
Feature : CONTRIBUTOR-TYPE_unique_values ------- Number of Unique Categories : 99 
Feature : OWNERSHIP-IND_Top_value ------- Number of Unique Categories : 4 
Feature : OWNERSHIP-IND_unique_values ------- Number of Unique Categories : 13 
Feature : ACCOUNT-STATUS_Top_value ------- Number of Unique Categories : 6 
Feature : ACCOUNT-STATUS_unique_values ------- Number of Unique Categories : 44 


In [ ]:
train_bur_demo.shape

(128654, 53)

In [ ]:
test_bur_demo.shape

(14745, 52)

In [ ]:
final_df = pd.concat([train_bur_demo,test_bur_demo],axis=0)
final_df.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1.0,0,1,0,1,1,1,1,9,True,5,True False,PRIMARY,9,PRIMARY,4,Auto Loan (Personal),5,"['Auto Loan (Personal)', 'Credit Card', 'Overd...",2,NBF,5,"['NAB', 'NBF']",1,Individual,9,['Individual'],3,Closed,5,"['Active', 'Closed', 'Delinquent']",2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0.0,0,1,0,0,1,1,2,13,False,12,True False,PRIMARY,13,PRIMARY,7,Commercial Vehicle Loan,4,"['Auto Loan (Personal)', 'Business Loan Genera...",2,NBF,7,"['NBF', 'PRB']",2,Individual,9,"['Guarantor', 'Individual']",2,Active,7,"['Active', 'Closed']",1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6.0,0,0,1,1,0,1,3,31,False,29,True False,PRIMARY,31,PRIMARY,8,Gold Loan,8,"['Commercial Vehicle Loan', 'Consumer Loan', '...",5,PRB,13,"['CCC', 'COP', 'NAB', 'NBF', 'PRB']",3,Individual,26,"['Guarantor', 'Individual', 'Joint']",3,Closed,19,"['Active', 'Closed', 'Delinquent']",1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1.0,0,1,0,0,1,1,7,4,True,2,True False,PRIMARY,4,PRIMARY,2,Tractor Loan,3,"['Personal Loan', 'Tractor Loan']",1,NBF,4,['NBF'],1,Individual,4,['Individual'],2,Closed,2,"['Active', 'Closed']",3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2.0,0,1,0,1,1,1,8,7,False,6,True False,PRIMARY,7,PRIMARY,3,Tractor Loan,4,"['Commercial Vehicle Loan', 'Personal Loan', '...",3,NBF,4,"['NAB', 'NBF', 'PRB']",3,Individual,3,"['Guarantor', 'Individual', 'Joint']",4,Active,3,"['Active', 'Closed', 'Delinquent', 'SUIT FILED...",2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524


In [ ]:
final_df.index = range(0,len(final_df))

In [ ]:
for feature in categorical_features:
  print('Feature : {} ------- Number of Unique Categories : {} '.format(feature,len(final_df[feature].unique())))

Feature : SELF-INDICATOR_Top_value ------- Number of Unique Categories : 2 
Feature : SELF-INDICATOR_unique_values ------- Number of Unique Categories : 3 
Feature : MATCH-TYPE_Top_value ------- Number of Unique Categories : 2 
Feature : MATCH-TYPE_unique_values ------- Number of Unique Categories : 3 
Feature : ACCT-TYPE_Top_value ------- Number of Unique Categories : 41 
Feature : ACCT-TYPE_unique_values ------- Number of Unique Categories : 8349 
Feature : CONTRIBUTOR-TYPE_Top_value ------- Number of Unique Categories : 10 
Feature : CONTRIBUTOR-TYPE_unique_values ------- Number of Unique Categories : 186 
Feature : OWNERSHIP-IND_Top_value ------- Number of Unique Categories : 5 
Feature : OWNERSHIP-IND_unique_values ------- Number of Unique Categories : 18 
Feature : ACCOUNT-STATUS_Top_value ------- Number of Unique Categories : 9 
Feature : ACCOUNT-STATUS_unique_values ------- Number of Unique Categories : 92 


### Categorical Encoding based on RANK by value count 

In [ ]:
final_dff = final_df.copy()

In [ ]:
def cat_enc_rank_value_count(features,final_dff):
  for feature in features:
    rank_acc_type_top = final_dff[feature].value_counts().to_frame()
    rank_acc_type_top[feature] = range(1,len(rank_acc_type_top)+1)
    rank_acc_type_dict = rank_acc_type_top[feature].to_dict()
    final_dff[feature] = final_dff[feature].map(rank_acc_type_dict)
  return final_dff

In [ ]:
final_df_acc_top = cat_enc_rank_value_count(categorical_features,final_dff)

In [ ]:
final_df_acc_top.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1.0,0,1,0,1,1,1,1,9,1,5,1,1,9,1,4,7,5,495,2,1,5,2,1,1,9,1,3,1,5,4,2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0.0,0,1,0,0,1,1,2,13,2,12,1,1,13,1,7,10,4,4006,2,1,7,3,2,1,9,3,2,2,7,1,1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6.0,0,0,1,1,0,1,3,31,2,29,1,1,31,1,8,2,8,2899,5,3,13,54,3,1,26,4,3,1,19,4,1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1.0,0,1,0,0,1,1,7,4,1,2,1,1,4,1,2,1,3,19,1,1,4,1,1,1,4,1,2,1,2,1,3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2.0,0,1,0,1,1,1,8,7,2,6,1,1,7,1,3,1,4,162,3,1,4,4,3,1,3,4,4,2,3,44,2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524


In [ ]:
train_data_final = final_df_acc_top.iloc[:128654,:]
test_data_final = final_df_acc_top.iloc[128654:,:]
display(train_data_final.head())
display(test_data_final.head())

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1.0,0,1,0,1,1,1,1,9,1,5,1,1,9,1,4,7,5,495,2,1,5,2,1,1,9,1,3,1,5,4,2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0.0,0,1,0,0,1,1,2,13,2,12,1,1,13,1,7,10,4,4006,2,1,7,3,2,1,9,3,2,2,7,1,1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6.0,0,0,1,1,0,1,3,31,2,29,1,1,31,1,8,2,8,2899,5,3,13,54,3,1,26,4,3,1,19,4,1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1.0,0,1,0,0,1,1,7,4,1,2,1,1,4,1,2,1,3,19,1,1,4,1,1,1,4,1,2,1,2,1,3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2.0,0,1,0,1,1,1,8,7,2,6,1,1,7,1,3,1,4,162,3,1,4,4,3,1,3,4,4,2,3,44,2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524


,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
128654,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,NaN,0,1,0,0,1,1,4,4,2,3,1,1,4,1,4,1,1,2683,3,6,2,79,2,1,2,2,2,2,3,1,741147.000000,8005.0,547818.500000,0.0,0.0,60.000000,1.403017,641864.840640,651.864532,0.0
128655,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,NaN,0,1,0,0,1,1,5,8,2,7,1,1,8,1,5,2,3,1251,4,1,3,65,2,1,6,2,2,1,6,1,242476.625000,1282.5,113403.875000,0.0,0.0,22.500000,2.127743,211590.023324,789.747186,0.0
128656,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,NaN,0,0,1,1,1,1,6,12,2,10,1,1,12,1,4,7,7,310,2,1,11,3,3,1,6,4,2,1,11,1,269780.083333,0.0,8566.666667,0.0,0.0,14.083333,6.326831,66122.300453,1864.970541,0.0
128657,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,NaN,1,0,0,1,1,1,25,2,1,1,1,1,2,1,2,3,1,2,2,1,1,2,1,1,2,1,2,2,1,1,400000.000000,0.0,63796.500000,0.0,0.0,0.000000,0.000000,163930.644118,0.000000,0.0
128658,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,NaN,0,0,1,1,1,1,119,2,1,1,1,1,2,1,2,7,1,5,1,1,2,1,2,2,1,3,2,2,1,1,465000.000000,8000.0,58100.000000,0.0,0.0,23.000000,5.388889,253020.115873,158.800000,265000.0


In [ ]:
train_data_final['top_up_class'] = train_data_final['top_up_class'].astype(int)
test_data_final.index = range(0,len(test_data_final))
test_data_final = test_data_final.drop('top_up_class',axis=1)
display(train_data_final.head())
display(test_data_final.head())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1,0,1,0,1,1,1,1,9,1,5,1,1,9,1,4,7,5,495,2,1,5,2,1,1,9,1,3,1,5,4,2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0,0,1,0,0,1,1,2,13,2,12,1,1,13,1,7,10,4,4006,2,1,7,3,2,1,9,3,2,2,7,1,1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6,0,0,1,1,0,1,3,31,2,29,1,1,31,1,8,2,8,2899,5,3,13,54,3,1,26,4,3,1,19,4,1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1,0,1,0,0,1,1,7,4,1,2,1,1,4,1,2,1,3,19,1,1,4,1,1,1,4,1,2,1,2,1,3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2,0,1,0,1,1,1,8,7,2,6,1,1,7,1,3,1,4,162,3,1,4,4,3,1,3,4,4,2,3,44,2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524


,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,4,2,3,1,1,4,1,4,1,1,2683,3,6,2,79,2,1,2,2,2,2,3,1,741147.000000,8005.0,547818.500000,0.0,0.0,60.000000,1.403017,641864.840640,651.864532,0.0
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,8,2,7,1,1,8,1,5,2,3,1251,4,1,3,65,2,1,6,2,2,1,6,1,242476.625000,1282.5,113403.875000,0.0,0.0,22.500000,2.127743,211590.023324,789.747186,0.0
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,12,2,10,1,1,12,1,4,7,7,310,2,1,11,3,3,1,6,4,2,1,11,1,269780.083333,0.0,8566.666667,0.0,0.0,14.083333,6.326831,66122.300453,1864.970541,0.0
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,2,1,1,1,1,2,1,2,3,1,2,2,1,1,2,1,1,2,1,2,2,1,1,400000.000000,0.0,63796.500000,0.0,0.0,0.000000,0.000000,163930.644118,0.000000,0.0
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,2,1,1,1,1,2,1,2,7,1,5,1,1,2,1,2,2,1,3,2,2,1,1,465000.000000,8000.0,58100.000000,0.0,0.0,23.000000,5.388889,253020.115873,158.800000,265000.0


In [ ]:
print(train_data_final.shape)
print(test_data_final.shape)

(128654, 53)
(14745, 52)


In [ ]:
train_data_final.to_excel('train_data_final_bur_combined_rank_cat_enc_mean.xlsx')
test_data_final.to_excel('test_data_final_bur_combined_rank_cat_enc_mean.xlsx')

### Categorical Encoding based on Rank Top10 Dummiers

In [ ]:
for feature in categorical_features:
  print('Feature : {} ------- Number of Unique Categories : {} '.format(feature,len(final_df[feature].unique())))

Feature : SELF-INDICATOR_Top_value ------- Number of Unique Categories : 2 
Feature : SELF-INDICATOR_unique_values ------- Number of Unique Categories : 3 
Feature : MATCH-TYPE_Top_value ------- Number of Unique Categories : 2 
Feature : MATCH-TYPE_unique_values ------- Number of Unique Categories : 3 
Feature : ACCT-TYPE_Top_value ------- Number of Unique Categories : 41 
Feature : ACCT-TYPE_unique_values ------- Number of Unique Categories : 8349 
Feature : CONTRIBUTOR-TYPE_Top_value ------- Number of Unique Categories : 10 
Feature : CONTRIBUTOR-TYPE_unique_values ------- Number of Unique Categories : 186 
Feature : OWNERSHIP-IND_Top_value ------- Number of Unique Categories : 5 
Feature : OWNERSHIP-IND_unique_values ------- Number of Unique Categories : 18 
Feature : ACCOUNT-STATUS_Top_value ------- Number of Unique Categories : 9 
Feature : ACCOUNT-STATUS_unique_values ------- Number of Unique Categories : 92 


In [ ]:
final_dff = final_df.copy()
final_dff.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1.0,0,1,0,1,1,1,1,9,True,5,True False,PRIMARY,9,PRIMARY,4,Auto Loan (Personal),5,"['Auto Loan (Personal)', 'Credit Card', 'Overd...",2,NBF,5,"['NAB', 'NBF']",1,Individual,9,['Individual'],3,Closed,5,"['Active', 'Closed', 'Delinquent']",2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0.0,0,1,0,0,1,1,2,13,False,12,True False,PRIMARY,13,PRIMARY,7,Commercial Vehicle Loan,4,"['Auto Loan (Personal)', 'Business Loan Genera...",2,NBF,7,"['NBF', 'PRB']",2,Individual,9,"['Guarantor', 'Individual']",2,Active,7,"['Active', 'Closed']",1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6.0,0,0,1,1,0,1,3,31,False,29,True False,PRIMARY,31,PRIMARY,8,Gold Loan,8,"['Commercial Vehicle Loan', 'Consumer Loan', '...",5,PRB,13,"['CCC', 'COP', 'NAB', 'NBF', 'PRB']",3,Individual,26,"['Guarantor', 'Individual', 'Joint']",3,Closed,19,"['Active', 'Closed', 'Delinquent']",1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1.0,0,1,0,0,1,1,7,4,True,2,True False,PRIMARY,4,PRIMARY,2,Tractor Loan,3,"['Personal Loan', 'Tractor Loan']",1,NBF,4,['NBF'],1,Individual,4,['Individual'],2,Closed,2,"['Active', 'Closed']",3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2.0,0,1,0,1,1,1,8,7,False,6,True False,PRIMARY,7,PRIMARY,3,Tractor Loan,4,"['Commercial Vehicle Loan', 'Personal Loan', '...",3,NBF,4,"['NAB', 'NBF', 'PRB']",3,Individual,3,"['Guarantor', 'Individual', 'Joint']",4,Active,3,"['Active', 'Closed', 'Delinquent', 'SUIT FILED...",2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524


In [ ]:
def cat_enc_rank_value_count(features,final_dff):
  for feature in features:
    rank_acc_type_top = final_dff[feature].value_counts().to_frame()
    rank_acc_type_top[feature] = range(1,len(rank_acc_type_top)+1)
    rank_acc_type_dict = rank_acc_type_top[feature].to_dict()
    final_dff[feature] = final_dff[feature].map(rank_acc_type_dict)
  return final_dff

final_df_acc_top = cat_enc_rank_value_count(['ACCT-TYPE_Top_value','CONTRIBUTOR-TYPE_unique_values','OWNERSHIP-IND_unique_values','ACCOUNT-STATUS_unique_values'],final_dff)

### Encoding ACCT-TYPE_unique_values in Top 10 most frequent labels

In [ ]:
final_df_acc_top['ACCT-TYPE_unique_values'].value_counts().sort_values(ascending=False).head(20) ##In Kdd they took only top 10 for remaining it is 0

['Tractor Loan']                                                                                    56058
['Business Loan Priority Sector  Agriculture', 'Tractor Loan']                                       7102
['Kisan Credit Card', 'Tractor Loan']                                                                5394
['Business Loan Priority Sector  Agriculture', 'Kisan Credit Card', 'Tractor Loan']                  3745
['Auto Loan (Personal)', 'Tractor Loan']                                                             3332
['Tractor Loan', 'Two-Wheeler Loan']                                                                 3249
['Gold Loan', 'Tractor Loan']                                                                        3105
['Business Loan Priority Sector  Agriculture']                                                       3076
['Other', 'Tractor Loan']                                                                            1922
['Commercial Vehicle Loan', 'Tractor Loan']   

In [ ]:
final_df_acc_top['ACCT-TYPE_unique_values'].value_counts().sort_values(ascending=False).head(20).index

Index(['['Tractor Loan']',
       '['Business Loan Priority Sector  Agriculture', 'Tractor Loan']',
       '['Kisan Credit Card', 'Tractor Loan']',
       '['Business Loan Priority Sector  Agriculture', 'Kisan Credit Card', 'Tractor Loan']',
       '['Auto Loan (Personal)', 'Tractor Loan']',
       '['Tractor Loan', 'Two-Wheeler Loan']', '['Gold Loan', 'Tractor Loan']',
       '['Business Loan Priority Sector  Agriculture']',
       '['Other', 'Tractor Loan']',
       '['Commercial Vehicle Loan', 'Tractor Loan']',
       '['Business Loan Priority Sector  Agriculture', 'Gold Loan', 'Kisan Credit Card', 'Tractor Loan']',
       '['Business Loan Priority Sector  Agriculture', 'Gold Loan', 'Tractor Loan']',
       '['Credit Card', 'Tractor Loan']',
       '['Gold Loan', 'Kisan Credit Card', 'Tractor Loan']',
       '['Consumer Loan', 'Tractor Loan']', '['Other']',
       '['Auto Loan (Personal)', 'Business Loan Priority Sector  Agriculture', 'Tractor Loan']',
       '['Overdraft', 'Tractor

In [ ]:
top20 = [x for x in final_df_acc_top['ACCT-TYPE_unique_values'].value_counts().sort_values(ascending=False).head(20).index]
top20

["['Tractor Loan']",
 "['Business Loan Priority Sector  Agriculture', 'Tractor Loan']",
 "['Kisan Credit Card', 'Tractor Loan']",
 "['Business Loan Priority Sector  Agriculture', 'Kisan Credit Card', 'Tractor Loan']",
 "['Auto Loan (Personal)', 'Tractor Loan']",
 "['Tractor Loan', 'Two-Wheeler Loan']",
 "['Gold Loan', 'Tractor Loan']",
 "['Business Loan Priority Sector  Agriculture']",
 "['Other', 'Tractor Loan']",
 "['Commercial Vehicle Loan', 'Tractor Loan']",
 "['Business Loan Priority Sector  Agriculture', 'Gold Loan', 'Kisan Credit Card', 'Tractor Loan']",
 "['Business Loan Priority Sector  Agriculture', 'Gold Loan', 'Tractor Loan']",
 "['Credit Card', 'Tractor Loan']",
 "['Gold Loan', 'Kisan Credit Card', 'Tractor Loan']",
 "['Consumer Loan', 'Tractor Loan']",
 "['Other']",
 "['Auto Loan (Personal)', 'Business Loan Priority Sector  Agriculture', 'Tractor Loan']",
 "['Overdraft', 'Tractor Loan']",
 "['Personal Loan', 'Tractor Loan']",
 "['Business Loan Priority Sector  Agriculture

In [ ]:
for label in top20:
    #We will create a new column in data with 10 binary variables of top10 , use where as conditional operator 
    final_df_acc_top[label] = np.where(final_df_acc_top['ACCT-TYPE_unique_values']==label,1,0)

In [ ]:
final_df_acc_top.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean,['Tractor Loan'],"['Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Kisan Credit Card', 'Tractor Loan']","['Auto Loan (Personal)', 'Tractor Loan']","['Tractor Loan', 'Two-Wheeler Loan']","['Gold Loan', 'Tractor Loan']",['Business Loan Priority Sector Agriculture'],"['Other', 'Tractor Loan']","['Commercial Vehicle Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Tractor Loan']","['Credit Card', 'Tractor Loan']","['Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Consumer Loan', 'Tractor Loan']",['Other'],"['Auto Loan (Personal)', 'Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Overdraft', 'Tractor Loan']","['Personal Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Other', 'Tractor Loan']"
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1.0,0,1,0,1,1,1,1,9,True,5,True False,PRIMARY,9,PRIMARY,4,7,5,"['Auto Loan (Personal)', 'Credit Card', 'Overd...",2,NBF,5,2,1,Individual,9,1,3,Closed,5,4,2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0.0,0,1,0,0,1,1,2,13,False,12,True False,PRIMARY,13,PRIMARY,7,10,4,"['Auto Loan (Personal)', 'Business Loan Genera...",2,NBF,7,3,2,Individual,9,3,2,Active,7,1,1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6.0,0,0,1,1,0,1,3,31,False,29,True False,PRIMARY,31,PRIMARY,8,2,8,"['Commercial Vehicle Loan', 'Consumer Loan', '...",5,PRB,13,55,3,Individual,26,4,3,Closed,19,4,1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1.0,0,1,0,0,1,1,7,4,True,2,True False,PRIMARY,4,PRIMARY,2,1,3,"['Personal Loan', 'Tractor Loan']",1,NBF,4,1,1,Individual,4,1,2,Closed,2,1,3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2.0,0,1,0,1,1,1,8,7,False,6,True False,PRIMARY,7,PRIMARY,3,1,4,"['Commercial Vehicle Loan', 'Personal Loan', '...",3,NBF,4,4,3,Individual,3,4,4,Active,3,45,2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
final_df_acc_top[['ACCT-TYPE_unique_values']+top20].head(30)

,ACCT-TYPE_unique_values,['Tractor Loan'],"['Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Kisan Credit Card', 'Tractor Loan']","['Auto Loan (Personal)', 'Tractor Loan']","['Tractor Loan', 'Two-Wheeler Loan']","['Gold Loan', 'Tractor Loan']",['Business Loan Priority Sector Agriculture'],"['Other', 'Tractor Loan']","['Commercial Vehicle Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Tractor Loan']","['Credit Card', 'Tractor Loan']","['Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Consumer Loan', 'Tractor Loan']",['Other'],"['Auto Loan (Personal)', 'Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Overdraft', 'Tractor Loan']","['Personal Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Other', 'Tractor Loan']"
0,"['Auto Loan (Personal)', 'Credit Card', 'Overd...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"['Auto Loan (Personal)', 'Business Loan Genera...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"['Commercial Vehicle Loan', 'Consumer Loan', '...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"['Personal Loan', 'Tractor Loan']",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,"['Commercial Vehicle Loan', 'Personal Loan', '...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,['Tractor Loan'],1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,"['Commercial Vehicle Loan', 'Tractor Loan']",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
7,"['Auto Loan (Personal)', 'Business Loan Genera...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,['Tractor Loan'],1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,"['Auto Loan (Personal)', 'Business Loan Priori...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
final_df_acc_top = final_df_acc_top.drop('ACCT-TYPE_unique_values',axis=1)
final_df_acc_top.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean,['Tractor Loan'],"['Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Kisan Credit Card', 'Tractor Loan']","['Auto Loan (Personal)', 'Tractor Loan']","['Tractor Loan', 'Two-Wheeler Loan']","['Gold Loan', 'Tractor Loan']",['Business Loan Priority Sector Agriculture'],"['Other', 'Tractor Loan']","['Commercial Vehicle Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Tractor Loan']","['Credit Card', 'Tractor Loan']","['Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Consumer Loan', 'Tractor Loan']",['Other'],"['Auto Loan (Personal)', 'Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Overdraft', 'Tractor Loan']","['Personal Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Other', 'Tractor Loan']"
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1.0,0,1,0,1,1,1,1,9,True,5,True False,PRIMARY,9,PRIMARY,4,7,5,2,NBF,5,2,1,Individual,9,1,3,Closed,5,4,2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0.0,0,1,0,0,1,1,2,13,False,12,True False,PRIMARY,13,PRIMARY,7,10,4,2,NBF,7,3,2,Individual,9,3,2,Active,7,1,1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6.0,0,0,1,1,0,1,3,31,False,29,True False,PRIMARY,31,PRIMARY,8,2,8,5,PRB,13,55,3,Individual,26,4,3,Closed,19,4,1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1.0,0,1,0,0,1,1,7,4,True,2,True False,PRIMARY,4,PRIMARY,2,1,3,1,NBF,4,1,1,Individual,4,1,2,Closed,2,1,3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2.0,0,1,0,1,1,1,8,7,False,6,True False,PRIMARY,7,PRIMARY,3,1,4,3,NBF,4,4,3,Individual,3,4,4,Active,3,45,2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
final_df_acc_top = pd.get_dummies(final_df_acc_top,drop_first=True)
final_df_acc_top.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value_freq,MATCH-TYPE_Top_value_freq,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean,['Tractor Loan'],"['Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Kisan Credit Card', 'Tractor Loan']","['Auto Loan (Personal)', 'Tractor Loan']","['Tractor Loan', 'Two-Wheeler Loan']","['Gold Loan', 'Tractor Loan']",['Business Loan Priority Sector Agriculture'],"['Other', 'Tractor Loan']","['Commercial Vehicle Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Tractor Loan']","['Credit Card', 'Tractor Loan']","['Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Consumer Loan', 'Tractor Loan']",['Other'],"['Auto Loan (Personal)', 'Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Overdraft', 'Tractor Loan']","['Personal Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Other', 'Tractor Loan']",SELF-INDICATOR_Top_value_True,SELF-INDICATOR_unique_values_True,SELF-INDICATOR_unique_values_True False,MATCH-TYPE_Top_value_SECONDARY,MATCH-TYPE_unique_values_PRIMARY SECONDARY,MATCH-TYPE_unique_values_SECONDARY,CONTRIBUTOR-TYPE_Top_value_COP,CONTRIBUTOR-TYPE_Top_value_FRB,CONTRIBUTOR-TYPE_Top_value_HFC,CONTRIBUTOR-TYPE_Top_value_MFI,CONTRIBUTOR-TYPE_Top_value_NAB,CONTRIBUTOR-TYPE_Top_value_NBF,CONTRIBUTOR-TYPE_Top_value_PRB,CONTRIBUTOR-TYPE_Top_value_RRB,CONTRIBUTOR-TYPE_Top_value_SFB,OWNERSHIP-IND_Top_value_Individual,OWNERSHIP-IND_Top_value_Joint,OWNERSHIP-IND_Top_value_Primary,OWNERSHIP-IND_Top_value_Supl Card Holder,ACCOUNT-STATUS_Top_value_Closed,ACCOUNT-STATUS_Top_value_Delinquent,ACCOUNT-STATUS_Top_value_Restructured,ACCOUNT-STATUS_Top_value_SUIT FILED (WILFUL DEFAULT),ACCOUNT-STATUS_Top_value_Settled,ACCOUNT-STATUS_Top_value_Suit Filed,ACCOUNT-STATUS_Top_value_WILFUL DEFAULT,ACCOUNT-STATUS_Top_value_Written Off
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1.0,0,1,0,1,1,1,1,9,5,9,4,7,5,2,5,2,1,9,1,3,5,4,2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0.0,0,1,0,0,1,1,2,13,12,13,7,10,4,2,7,3,2,9,3,2,7,1,1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6.0,0,0,1,1,0,1,3,31,29,31,8,2,8,5,13,55,3,26,4,3,19,4,1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1.0,0,1,0,0,1,1,7,4,2,4,2,1,3,1,4,1,1,4,1,2,2,1,3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+

In [ ]:
final_df_acc_top.shape

(143399, 92)

In [ ]:
train_data_final = final_df_acc_top.iloc[:128654,:]
test_data_final = final_df_acc_top.iloc[128654:,:]
display(train_data_final.head())
display(test_data_final.head())

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value_freq,MATCH-TYPE_Top_value_freq,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean,['Tractor Loan'],"['Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Kisan Credit Card', 'Tractor Loan']","['Auto Loan (Personal)', 'Tractor Loan']","['Tractor Loan', 'Two-Wheeler Loan']","['Gold Loan', 'Tractor Loan']",['Business Loan Priority Sector Agriculture'],"['Other', 'Tractor Loan']","['Commercial Vehicle Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Tractor Loan']","['Credit Card', 'Tractor Loan']","['Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Consumer Loan', 'Tractor Loan']",['Other'],"['Auto Loan (Personal)', 'Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Overdraft', 'Tractor Loan']","['Personal Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Other', 'Tractor Loan']",SELF-INDICATOR_Top_value_True,SELF-INDICATOR_unique_values_True,SELF-INDICATOR_unique_values_True False,MATCH-TYPE_Top_value_SECONDARY,MATCH-TYPE_unique_values_PRIMARY SECONDARY,MATCH-TYPE_unique_values_SECONDARY,CONTRIBUTOR-TYPE_Top_value_COP,CONTRIBUTOR-TYPE_Top_value_FRB,CONTRIBUTOR-TYPE_Top_value_HFC,CONTRIBUTOR-TYPE_Top_value_MFI,CONTRIBUTOR-TYPE_Top_value_NAB,CONTRIBUTOR-TYPE_Top_value_NBF,CONTRIBUTOR-TYPE_Top_value_PRB,CONTRIBUTOR-TYPE_Top_value_RRB,CONTRIBUTOR-TYPE_Top_value_SFB,OWNERSHIP-IND_Top_value_Individual,OWNERSHIP-IND_Top_value_Joint,OWNERSHIP-IND_Top_value_Primary,OWNERSHIP-IND_Top_value_Supl Card Holder,ACCOUNT-STATUS_Top_value_Closed,ACCOUNT-STATUS_Top_value_Delinquent,ACCOUNT-STATUS_Top_value_Restructured,ACCOUNT-STATUS_Top_value_SUIT FILED (WILFUL DEFAULT),ACCOUNT-STATUS_Top_value_Settled,ACCOUNT-STATUS_Top_value_Suit Filed,ACCOUNT-STATUS_Top_value_WILFUL DEFAULT,ACCOUNT-STATUS_Top_value_Written Off
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1.0,0,1,0,1,1,1,1,9,5,9,4,7,5,2,5,2,1,9,1,3,5,4,2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0.0,0,1,0,0,1,1,2,13,12,13,7,10,4,2,7,3,2,9,3,2,7,1,1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6.0,0,0,1,1,0,1,3,31,29,31,8,2,8,5,13,55,3,26,4,3,19,4,1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1.0,0,1,0,0,1,1,7,4,2,4,2,1,3,1,4,1,1,4,1,2,2,1,3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value_freq,MATCH-TYPE_Top_value_freq,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean,['Tractor Loan'],"['Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Kisan Credit Card', 'Tractor Loan']","['Auto Loan (Personal)', 'Tractor Loan']","['Tractor Loan', 'Two-Wheeler Loan']","['Gold Loan', 'Tractor Loan']",['Business Loan Priority Sector Agriculture'],"['Other', 'Tractor Loan']","['Commercial Vehicle Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Tractor Loan']","['Credit Card', 'Tractor Loan']","['Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Consumer Loan', 'Tractor Loan']",['Other'],"['Auto Loan (Personal)', 'Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Overdraft', 'Tractor Loan']","['Personal Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Other', 'Tractor Loan']",SELF-INDICATOR_Top_value_True,SELF-INDICATOR_unique_values_True,SELF-INDICATOR_unique_values_True False,MATCH-TYPE_Top_value_SECONDARY,MATCH-TYPE_unique_values_PRIMARY SECONDARY,MATCH-TYPE_unique_values_SECONDARY,CONTRIBUTOR-TYPE_Top_value_COP,CONTRIBUTOR-TYPE_Top_value_FRB,CONTRIBUTOR-TYPE_Top_value_HFC,CONTRIBUTOR-TYPE_Top_value_MFI,CONTRIBUTOR-TYPE_Top_value_NAB,CONTRIBUTOR-TYPE_Top_value_NBF,CONTRIBUTOR-TYPE_Top_value_PRB,CONTRIBUTOR-TYPE_Top_value_RRB,CONTRIBUTOR-TYPE_Top_value_SFB,OWNERSHIP-IND_Top_value_Individual,OWNERSHIP-IND_Top_value_Joint,OWNERSHIP-IND_Top_value_Primary,OWNERSHIP-IND_Top_value_Supl Card Holder,ACCOUNT-STATUS_Top_value_Closed,ACCOUNT-STATUS_Top_value_Delinquent,ACCOUNT-STATUS_Top_value_Restructured,ACCOUNT-STATUS_Top_value_SUIT FILED (WILFUL DEFAULT),ACCOUNT-STATUS_Top_value_Settled,ACCOUNT-STATUS_Top_value_Suit Filed,ACCOUNT-STATUS_Top_value_WILFUL DEFAULT,ACCOUNT-STATUS_Top_value_Written Off
128654,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,NaN,0,1,0,0,1,1,4,4,3,4,4,1,1,3,2,83,2,2,2,2,3,1,741147.000000,8005.0,547818.500000,0.0,0.0,60.000000,1.403017,641864.840640,651.864532,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
128655,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,NaN,0,1,0,0,1,1,5,8,7,8,5,2,3,4,3,65,2,6,2,2,6,1,242476.625000,1282.5,113403.875000,0.0,0.0,22.500000,2.127743,211590.023324,789.747186,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
128656,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,NaN,0,0,1,1,1,1,6,12,10,12,4,7,7,2,11,3,3,6,4,2,11,1,269780.083333,0.0,8566.666667,0.0,0.0,14.083333,6.326831,66122.300453,1864.970541,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
128657,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,NaN,1,0,0,1,1,1,25,2,1,2,2,3,1,2,1,2,1,2,1,2,1,1,400000.000000,0.0,63796.500000,0.0,0.0,0.000000,0.000000,163930.644118,0.000000,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

In [ ]:
train_data_final['top_up_class'] = train_data_final['top_up_class'].astype(int)
test_data_final.index = range(0,len(test_data_final))
test_data_final = test_data_final.drop('top_up_class',axis=1)
display(train_data_final.head())
display(test_data_final.head())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value_freq,MATCH-TYPE_Top_value_freq,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean,['Tractor Loan'],"['Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Kisan Credit Card', 'Tractor Loan']","['Auto Loan (Personal)', 'Tractor Loan']","['Tractor Loan', 'Two-Wheeler Loan']","['Gold Loan', 'Tractor Loan']",['Business Loan Priority Sector Agriculture'],"['Other', 'Tractor Loan']","['Commercial Vehicle Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Tractor Loan']","['Credit Card', 'Tractor Loan']","['Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Consumer Loan', 'Tractor Loan']",['Other'],"['Auto Loan (Personal)', 'Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Overdraft', 'Tractor Loan']","['Personal Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Other', 'Tractor Loan']",SELF-INDICATOR_Top_value_True,SELF-INDICATOR_unique_values_True,SELF-INDICATOR_unique_values_True False,MATCH-TYPE_Top_value_SECONDARY,MATCH-TYPE_unique_values_PRIMARY SECONDARY,MATCH-TYPE_unique_values_SECONDARY,CONTRIBUTOR-TYPE_Top_value_COP,CONTRIBUTOR-TYPE_Top_value_FRB,CONTRIBUTOR-TYPE_Top_value_HFC,CONTRIBUTOR-TYPE_Top_value_MFI,CONTRIBUTOR-TYPE_Top_value_NAB,CONTRIBUTOR-TYPE_Top_value_NBF,CONTRIBUTOR-TYPE_Top_value_PRB,CONTRIBUTOR-TYPE_Top_value_RRB,CONTRIBUTOR-TYPE_Top_value_SFB,OWNERSHIP-IND_Top_value_Individual,OWNERSHIP-IND_Top_value_Joint,OWNERSHIP-IND_Top_value_Primary,OWNERSHIP-IND_Top_value_Supl Card Holder,ACCOUNT-STATUS_Top_value_Closed,ACCOUNT-STATUS_Top_value_Delinquent,ACCOUNT-STATUS_Top_value_Restructured,ACCOUNT-STATUS_Top_value_SUIT FILED (WILFUL DEFAULT),ACCOUNT-STATUS_Top_value_Settled,ACCOUNT-STATUS_Top_value_Suit Filed,ACCOUNT-STATUS_Top_value_WILFUL DEFAULT,ACCOUNT-STATUS_Top_value_Written Off
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1,0,1,0,1,1,1,1,9,5,9,4,7,5,2,5,2,1,9,1,3,5,4,2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0,0,1,0,0,1,1,2,13,12,13,7,10,4,2,7,3,2,9,3,2,7,1,1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6,0,0,1,1,0,1,3,31,29,31,8,2,8,5,13,55,3,26,4,3,19,4,1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1,0,1,0,0,1,1,7,4,2,4,2,1,3,1,4,1,1,4,1,2,2,1,3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value_freq,MATCH-TYPE_Top_value_freq,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean,['Tractor Loan'],"['Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Kisan Credit Card', 'Tractor Loan']","['Auto Loan (Personal)', 'Tractor Loan']","['Tractor Loan', 'Two-Wheeler Loan']","['Gold Loan', 'Tractor Loan']",['Business Loan Priority Sector Agriculture'],"['Other', 'Tractor Loan']","['Commercial Vehicle Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Gold Loan', 'Tractor Loan']","['Credit Card', 'Tractor Loan']","['Gold Loan', 'Kisan Credit Card', 'Tractor Loan']","['Consumer Loan', 'Tractor Loan']",['Other'],"['Auto Loan (Personal)', 'Business Loan Priority Sector Agriculture', 'Tractor Loan']","['Overdraft', 'Tractor Loan']","['Personal Loan', 'Tractor Loan']","['Business Loan Priority Sector Agriculture', 'Other', 'Tractor Loan']",SELF-INDICATOR_Top_value_True,SELF-INDICATOR_unique_values_True,SELF-INDICATOR_unique_values_True False,MATCH-TYPE_Top_value_SECONDARY,MATCH-TYPE_unique_values_PRIMARY SECONDARY,MATCH-TYPE_unique_values_SECONDARY,CONTRIBUTOR-TYPE_Top_value_COP,CONTRIBUTOR-TYPE_Top_value_FRB,CONTRIBUTOR-TYPE_Top_value_HFC,CONTRIBUTOR-TYPE_Top_value_MFI,CONTRIBUTOR-TYPE_Top_value_NAB,CONTRIBUTOR-TYPE_Top_value_NBF,CONTRIBUTOR-TYPE_Top_value_PRB,CONTRIBUTOR-TYPE_Top_value_RRB,CONTRIBUTOR-TYPE_Top_value_SFB,OWNERSHIP-IND_Top_value_Individual,OWNERSHIP-IND_Top_value_Joint,OWNERSHIP-IND_Top_value_Primary,OWNERSHIP-IND_Top_value_Supl Card Holder,ACCOUNT-STATUS_Top_value_Closed,ACCOUNT-STATUS_Top_value_Delinquent,ACCOUNT-STATUS_Top_value_Restructured,ACCOUNT-STATUS_Top_value_SUIT FILED (WILFUL DEFAULT),ACCOUNT-STATUS_Top_value_Settled,ACCOUNT-STATUS_Top_value_Suit Filed,ACCOUNT-STATUS_Top_value_WILFUL DEFAULT,ACCOUNT-STATUS_Top_value_Written Off
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,4,3,4,4,1,1,3,2,83,2,2,2,2,3,1,741147.000000,8005.0,547818.500000,0.0,0.0,60.000000,1.403017,641864.840640,651.864532,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,8,7,8,5,2,3,4,3,65,2,6,2,2,6,1,242476.625000,1282.5,113403.875000,0.0,0.0,22.500000,2.127743,211590.023324,789.747186,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,12,10,12,4,7,7,2,11,3,3,6,4,2,11,1,269780.083333,0.0,8566.666667,0.0,0.0,14.083333,6.326831,66122.300453,1864.970541,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,2,1,2,2,3,1,2,1,2,1,2,1,2,1,1,400000.000000,0.0,63796.500000,0.0,0.0,0.000000,0.000000,163930.644118,0.000000,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,

In [ ]:
print(train_data_final.shape)
print(test_data_final.shape)

(128654, 92)
(14745, 91)


In [ ]:
train_data_final.to_excel('train_data_final_bur_combined_rank_top_dummies_cat_enc_mean.xlsx')
test_data_final.to_excel('test_data_final_bur_combined_rank_top_dummies_cat_enc_mean.xlsx')

#### Categorical Encoding of Value count 

In [ ]:
final_dff = final_df.copy()

In [ ]:
categorical_features

['SELF-INDICATOR_Top_value',
 'SELF-INDICATOR_unique_values',
 'MATCH-TYPE_Top_value',
 'MATCH-TYPE_unique_values',
 'ACCT-TYPE_Top_value',
 'ACCT-TYPE_unique_values',
 'CONTRIBUTOR-TYPE_Top_value',
 'CONTRIBUTOR-TYPE_unique_values',
 'OWNERSHIP-IND_Top_value',
 'OWNERSHIP-IND_unique_values',
 'ACCOUNT-STATUS_Top_value',
 'ACCOUNT-STATUS_unique_values']

In [ ]:
def cat_enc_rank_value_count(features,final_dff):
  for feature in features:
    rank_acc_type_top = final_dff[feature].value_counts().to_frame()
    rank_acc_type_dict = rank_acc_type_top[feature].to_dict()
    final_dff[feature] = final_dff[feature].map(rank_acc_type_dict)
  return final_dff

final_df_acc_top = cat_enc_rank_value_count(['ACCT-TYPE_Top_value','ACCT-TYPE_unique_values','CONTRIBUTOR-TYPE_Top_value','CONTRIBUTOR-TYPE_unique_values','OWNERSHIP-IND_unique_values','ACCOUNT-STATUS_unique_values'],final_dff)

In [ ]:
final_df_acc_top.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1.0,0,1,0,1,1,1,1,9,True,5,True False,PRIMARY,9,PRIMARY,4,3140,5,13,2,95479,5,26367,1,Individual,9,104533,3,Closed,5,16610,2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0.0,0,1,0,0,1,1,2,13,False,12,True False,PRIMARY,13,PRIMARY,7,1961,4,1,2,95479,7,13997,2,Individual,9,14151,2,Active,7,47116,1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6.0,0,0,1,1,0,1,3,31,False,29,True False,PRIMARY,31,PRIMARY,8,13746,8,1,5,6842,13,26,3,Individual,26,7053,3,Closed,19,16610,1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1.0,0,1,0,0,1,1,7,4,True,2,True False,PRIMARY,4,PRIMARY,2,80864,3,661,1,95479,4,68493,1,Individual,4,104533,2,Closed,2,47116,3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2.0,0,1,0,1,1,1,8,7,False,6,True False,PRIMARY,7,PRIMARY,3,80864,4,50,3,95479,4,11477,3,Individual,3,7053,4,Active,3,10,2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524


In [ ]:
final_df_acc_top = pd.get_dummies(final_df_acc_top,drop_first=True)
final_df_acc_top.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value_freq,MATCH-TYPE_Top_value_freq,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean,SELF-INDICATOR_Top_value_True,SELF-INDICATOR_unique_values_True,SELF-INDICATOR_unique_values_True False,MATCH-TYPE_Top_value_SECONDARY,MATCH-TYPE_unique_values_PRIMARY SECONDARY,MATCH-TYPE_unique_values_SECONDARY,OWNERSHIP-IND_Top_value_Individual,OWNERSHIP-IND_Top_value_Joint,OWNERSHIP-IND_Top_value_Primary,OWNERSHIP-IND_Top_value_Supl Card Holder,ACCOUNT-STATUS_Top_value_Closed,ACCOUNT-STATUS_Top_value_Delinquent,ACCOUNT-STATUS_Top_value_Restructured,ACCOUNT-STATUS_Top_value_SUIT FILED (WILFUL DEFAULT),ACCOUNT-STATUS_Top_value_Settled,ACCOUNT-STATUS_Top_value_Suit Filed,ACCOUNT-STATUS_Top_value_WILFUL DEFAULT,ACCOUNT-STATUS_Top_value_Written Off
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1.0,0,1,0,1,1,1,1,9,5,9,4,3140,5,13,2,95479,5,26367,1,9,104533,3,5,16610,2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0.0,0,1,0,0,1,1,2,13,12,13,7,1961,4,1,2,95479,7,13997,2,9,14151,2,7,47116,1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6.0,0,0,1,1,0,1,3,31,29,31,8,13746,8,1,5,6842,13,26,3,26,7053,3,19,16610,1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1.0,0,1,0,0,1,1,7,4,2,4,2,80864,3,661,1,95479,4,68493,1,4,104533,2,2,47116,3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2.0,0,1,0,1,1,1,8,7,6,7,3,80864,4,50,3,95479,4,11477,3,3,7053,4,3,10,2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
final_df_acc_top.shape

(143399, 65)

In [ ]:
train_data_final = final_df_acc_top.iloc[:128654,:]
test_data_final = final_df_acc_top.iloc[128654:,:]
display(train_data_final.head())
display(test_data_final.head())

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value_freq,MATCH-TYPE_Top_value_freq,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean,SELF-INDICATOR_Top_value_True,SELF-INDICATOR_unique_values_True,SELF-INDICATOR_unique_values_True False,MATCH-TYPE_Top_value_SECONDARY,MATCH-TYPE_unique_values_PRIMARY SECONDARY,MATCH-TYPE_unique_values_SECONDARY,OWNERSHIP-IND_Top_value_Individual,OWNERSHIP-IND_Top_value_Joint,OWNERSHIP-IND_Top_value_Primary,OWNERSHIP-IND_Top_value_Supl Card Holder,ACCOUNT-STATUS_Top_value_Closed,ACCOUNT-STATUS_Top_value_Delinquent,ACCOUNT-STATUS_Top_value_Restructured,ACCOUNT-STATUS_Top_value_SUIT FILED (WILFUL DEFAULT),ACCOUNT-STATUS_Top_value_Settled,ACCOUNT-STATUS_Top_value_Suit Filed,ACCOUNT-STATUS_Top_value_WILFUL DEFAULT,ACCOUNT-STATUS_Top_value_Written Off
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1.0,0,1,0,1,1,1,1,9,5,9,4,3140,5,13,2,95479,5,26367,1,9,104533,3,5,16610,2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0.0,0,1,0,0,1,1,2,13,12,13,7,1961,4,1,2,95479,7,13997,2,9,14151,2,7,47116,1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6.0,0,0,1,1,0,1,3,31,29,31,8,13746,8,1,5,6842,13,26,3,26,7053,3,19,16610,1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1.0,0,1,0,0,1,1,7,4,2,4,2,80864,3,661,1,95479,4,68493,1,4,104533,2,2,47116,3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2.0,0,1,0,1,1,1,8,7,6,7,3,80864,4,50,3,95479,4,11477,3,3,7053,4,3,10,2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value_freq,MATCH-TYPE_Top_value_freq,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean,SELF-INDICATOR_Top_value_True,SELF-INDICATOR_unique_values_True,SELF-INDICATOR_unique_values_True False,MATCH-TYPE_Top_value_SECONDARY,MATCH-TYPE_unique_values_PRIMARY SECONDARY,MATCH-TYPE_unique_values_SECONDARY,OWNERSHIP-IND_Top_value_Individual,OWNERSHIP-IND_Top_value_Joint,OWNERSHIP-IND_Top_value_Primary,OWNERSHIP-IND_Top_value_Supl Card Holder,ACCOUNT-STATUS_Top_value_Closed,ACCOUNT-STATUS_Top_value_Delinquent,ACCOUNT-STATUS_Top_value_Restructured,ACCOUNT-STATUS_Top_value_SUIT FILED (WILFUL DEFAULT),ACCOUNT-STATUS_Top_value_Settled,ACCOUNT-STATUS_Top_value_Suit Filed,ACCOUNT-STATUS_Top_value_WILFUL DEFAULT,ACCOUNT-STATUS_Top_value_Written Off
128654,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,NaN,0,1,0,0,1,1,4,4,3,4,4,80864,1,2,3,362,2,9,2,2,15784,2,3,47116,741147.000000,8005.0,547818.500000,0.0,0.0,60.000000,1.403017,641864.840640,651.864532,0.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
128655,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,NaN,0,1,0,0,1,1,5,8,7,8,5,13746,3,4,4,95479,3,16,2,6,15784,2,6,47116,242476.625000,1282.5,113403.875000,0.0,0.0,22.500000,2.127743,211590.023324,789.747186,0.0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
128656,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,NaN,0,0,1,1,1,1,6,12,10,12,4,3140,7,23,2,95479,11,13997,3,6,7053,2,11,47116,269780.083333,0.0,8566.666667,0.0,0.0,14.083333,6.326831,66122.300453,1864.970541,0.0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
128657,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,NaN,1,0,0,1,1,1,25,2,1,2,2,13554,1,7102,2,95479,1,26367,1,2,104533,2,1,47116,400000.000000,0.0,63796.500000,0.0,0.0,0.000000,0.000000,163930.644118,0.000000,0.0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
128658,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,NaN,0,0,1,1,1,1,119,2,1,2,2,3140,1,3332,1,95479,2,68493,2,1,14151,2,1,47116,465000.000000,8000.0,58100.000000,0.0,0.0,23.000000,5.388889,253020.115873,158.800000,265000.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_data_final['top_up_class'] = train_data_final['top_up_class'].astype(int)
test_data_final.index = range(0,len(test_data_final))
test_data_final = test_data_final.drop('top_up_class',axis=1)
display(train_data_final.head())
display(test_data_final.head())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value_freq,MATCH-TYPE_Top_value_freq,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean,SELF-INDICATOR_Top_value_True,SELF-INDICATOR_unique_values_True,SELF-INDICATOR_unique_values_True False,MATCH-TYPE_Top_value_SECONDARY,MATCH-TYPE_unique_values_PRIMARY SECONDARY,MATCH-TYPE_unique_values_SECONDARY,OWNERSHIP-IND_Top_value_Individual,OWNERSHIP-IND_Top_value_Joint,OWNERSHIP-IND_Top_value_Primary,OWNERSHIP-IND_Top_value_Supl Card Holder,ACCOUNT-STATUS_Top_value_Closed,ACCOUNT-STATUS_Top_value_Delinquent,ACCOUNT-STATUS_Top_value_Restructured,ACCOUNT-STATUS_Top_value_SUIT FILED (WILFUL DEFAULT),ACCOUNT-STATUS_Top_value_Settled,ACCOUNT-STATUS_Top_value_Suit Filed,ACCOUNT-STATUS_Top_value_WILFUL DEFAULT,ACCOUNT-STATUS_Top_value_Written Off
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1,0,1,0,1,1,1,1,9,5,9,4,3140,5,13,2,95479,5,26367,1,9,104533,3,5,16610,2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0,0,1,0,0,1,1,2,13,12,13,7,1961,4,1,2,95479,7,13997,2,9,14151,2,7,47116,1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6,0,0,1,1,0,1,3,31,29,31,8,13746,8,1,5,6842,13,26,3,26,7053,3,19,16610,1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1,0,1,0,0,1,1,7,4,2,4,2,80864,3,661,1,95479,4,68493,1,4,104533,2,2,47116,3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2,0,1,0,1,1,1,8,7,6,7,3,80864,4,50,3,95479,4,11477,3,3,7053,4,3,10,2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value_freq,MATCH-TYPE_Top_value_freq,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean,SELF-INDICATOR_Top_value_True,SELF-INDICATOR_unique_values_True,SELF-INDICATOR_unique_values_True False,MATCH-TYPE_Top_value_SECONDARY,MATCH-TYPE_unique_values_PRIMARY SECONDARY,MATCH-TYPE_unique_values_SECONDARY,OWNERSHIP-IND_Top_value_Individual,OWNERSHIP-IND_Top_value_Joint,OWNERSHIP-IND_Top_value_Primary,OWNERSHIP-IND_Top_value_Supl Card Holder,ACCOUNT-STATUS_Top_value_Closed,ACCOUNT-STATUS_Top_value_Delinquent,ACCOUNT-STATUS_Top_value_Restructured,ACCOUNT-STATUS_Top_value_SUIT FILED (WILFUL DEFAULT),ACCOUNT-STATUS_Top_value_Settled,ACCOUNT-STATUS_Top_value_Suit Filed,ACCOUNT-STATUS_Top_value_WILFUL DEFAULT,ACCOUNT-STATUS_Top_value_Written Off
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,4,3,4,4,80864,1,2,3,362,2,9,2,2,15784,2,3,47116,741147.000000,8005.0,547818.500000,0.0,0.0,60.000000,1.403017,641864.840640,651.864532,0.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,8,7,8,5,13746,3,4,4,95479,3,16,2,6,15784,2,6,47116,242476.625000,1282.5,113403.875000,0.0,0.0,22.500000,2.127743,211590.023324,789.747186,0.0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,12,10,12,4,3140,7,23,2,95479,11,13997,3,6,7053,2,11,47116,269780.083333,0.0,8566.666667,0.0,0.0,14.083333,6.326831,66122.300453,1864.970541,0.0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,2,1,2,2,13554,1,7102,2,95479,1,26367,1,2,104533,2,1,47116,400000.000000,0.0,63796.500000,0.0,0.0,0.000000,0.000000,163930.644118,0.000000,0.0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,2,1,2,2,3140,1,3332,1,95479,2,68493,2,1,14151,2,1,47116,465000.000000,8000.0,58100.000000,0.0,0.0,23.000000,5.388889,253020.115873,158.800000,265000.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(train_data_final.shape)
print(test_data_final.shape)

(128654, 65)
(14745, 64)


In [ ]:
train_data_final.to_excel('train_data_final_bur_combined_only_value_count_cat_enc_mean.xlsx')
test_data_final.to_excel('test_data_final_bur_combined_only_value_count_cat_enc_mean.xlsx')